In [2]:
from glob import glob
import numpy as np

In [3]:
stripfiles = sorted(glob('meshes/frame*.obj'))
stripfiles = [fl for fl in stripfiles if 't1' not in fl]
print('    ', len(stripfiles))
print(stripfiles[0])
print(stripfiles[-1])

     49
meshes\frame31000_strip_o100_s20_666.obj
meshes\frame31000_strip_o99_s20_766.obj


In [4]:
def parsev(line, comment=True, dtype=float):
    if comment: line = line.replace('# ', '')
    parts = line.split()
    token = parts[0]
    vlist = [dtype(val) for val in parts[1:]]
    return vlist, token

parsev('# asdf 1 2 3 4')

([1.0, 2.0, 3.0, 4.0], 'asdf')

In [5]:
RSTRIDE = 3
BSTART = 40
def load_markup(fname, ventries=4):
    with open(fname) as fl:
        raw = fl.read()
    lines = [ln for ln in raw.split('\n') if ln]
    registers = lines[2:2+256]
    registers = [parsev(ln)[0] for ln in registers]
    
    vstart, vend = -1, -1
    for lii in range(len(lines)):
        if lines[lii][0] == 'v' and vstart == -1: 
            vstart = lii + 1 - ventries
        if lines[lii][0] == 'f' and vend == -1: 
            vend = lii
    vgrouped = [lines[lii:lii+ventries] for lii in range(vstart, vend, ventries)]
    faces = [parsev(ln, comment=False, dtype=int)[0] for ln in lines if ln[0] == 'f']
    
    verts = []
    for group in vgrouped:
        ent = {}
        converted = [parsev(line) for line in group[:ventries-1]]
        for vector, token in converted: ent[token] = vector
        
        vertex, _ = parsev(group[-1], comment=False)
        ent['vertex'] = vertex + [1,]
        verts.append(ent)
    
    blended = []
    for ent in verts:
        vertex = ent['vertex']
        
        bind = int(ent['blend'][0])
        assert bind >= 0
        boffset = BSTART + RSTRIDE * bind
        
#         print('     ', vertex)
        bvect = np.zeros(4)
        for dim in range(3):
            bweights = np.array(registers[boffset+dim])
#             bweights /= bweights[-1]
#             print('     ', bweights)
            bvect[dim] = np.dot(vertex, bweights)
#         print('     ', bvect)
        bvect[3] = 1
        ent['blended'] = bvect
#         break
    
    return verts, faces, registers

verts, faces, regs = load_markup(stripfiles[0])
print(verts[0]['blended'])
print(verts[-1]['blended'])
print(faces[0])

[ 1.68073085e+03  9.99321440e+01 -1.57427949e+03  1.00000000e+00]
[ 1.63069866e+03  6.44377677e+01 -1.57639673e+03  1.00000000e+00]
[1, 1, 2]


In [6]:
def writeobj(fname, verts, faces):
    with open(fname, 'w') as fl:
        for ent in verts:
            fl.write('v %s\n' % ' '.join(['%f' % val for val in ent['blended'][:3]]))
        for fdef in faces:
            fl.write('f %s\n' % ' '.join(['%d' % val for val in fdef]))

stripfiles = sorted(glob('meshes/frame*.obj'))
stripfiles = [fl for fl in stripfiles if 't1' not in fl]

for fname in stripfiles:
    verts, faces, regs = load_markup(fname)
    fname = fname.replace('frame', 'blendf')
#     print(fname)
    writeobj(fname, verts, faces)

In [19]:
markupfiles = sorted(glob('meshes/frame*.obj'))
print(len(markupfiles))

class merged:
    verts = []
    faces = []
    
foffset = 0
for fname in markupfiles:
    verts, faces, regs = load_markup(fname)
    merged.verts += verts
    
    offset_faces = []
    for face in faces:
        offset_faces += [[val + foffset for val in face]]
    assert len(offset_faces) == len(faces)
    merged.faces += offset_faces
    
    foffset += len(verts)

with open('blended.obj', 'w') as fl:
    for vert in merged.verts:
#         print(vert)
        fl.write('v %s\n' % ' '.join(['%f' % val for val in vert['blended'][:3]]))
    for face in merged.faces:
        fl.write('f %s\n' % ' '.join(['%d' % val for val in face]))
        
with open('merged.obj', 'w') as fl:
    for vert in merged.verts:
#         print(vert)
        fl.write('v %s\n' % ' '.join(['%f' % val for val in vert['vertex'][:3]]))
    for face in merged.faces:
        fl.write('f %s\n' % ' '.join(['%d' % val for val in face]))
    

49
